In [ ]:
# This notebook is how to do a test that stops the telescope and lets stars drift through the field.

In [ ]:
# As a precondition, and not part of this notebook, the telescope system should be fully enabled and configured.

In [ ]:
import sys
import asyncio
import time
import os
from datetime import datetime

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [ ]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
#get catcs.rem.atmcs.evt_summaryState
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...

In [ ]:
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
tmp = await atcs.rem.atmcs.evt_summaryState.aget()
print(tmp)

In [ ]:
# enable components if required
# await atcs.standby()
# await latiss.standby()

In [ ]:
# make sure ATAOS corrections are on
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# Slew to target and setup instrument
await atcs.slew_object("HR 2934", rot_type=RotType.PhysicalSky, rot=-90)
await latiss.setup_atspec(filter='RG610', grating='empty_1')

In [ ]:
# take a quick image
await latiss.take_object(5)

In [ ]:
# offset in azimuth and take a quick image
az_offset=60 # arcsec
el_offset=0 # arcsec
await atcs.offset_azel(az_offset, el_offset)
await asyncio.sleep(2)
await latiss.take_object(5)

In [ ]:
# offset in elevation and take a quick image
az_offset=-60 # arcsec
el_offset=60 # arcsec
await atcs.offset_azel(az_offset, el_offset)
await asyncio.sleep(2)
await latiss.take_object(5)

In [ ]:
# Slew back target by resetting offsets then verify
await atcs.reset_offsets()
await asyncio.sleep(2)
await latiss.take_object(5)

In [ ]:
# Slew to target and setup instrument
await atcs.slew_object("HR 2934", rot_type=RotType.PhysicalSky, rot=-90)
await latiss.take_object(2)

In [ ]:
# Offset in RA, turn off tracking, start long exposure
ra_offset=-60*4 # 2 arcminute offset
dec_offset = 0
await atcs.offset_radec(ra_offset, dec_offset)
await asyncio.sleep(2)
# stop tracking, then start exposing

In [ ]:
#current_target = await atcs.rem.atptg.evt_currentTarget.aget(timeout=4) # isn't publishing alt-az! LAZY!
current_target = await atcs.rem.atptg.tel_currentTargetStatus.next(flush=True, timeout=4) # isn't publishing alt-az!

In [ ]:
print(current_target)

In [ ]:
await atcs.point_azel(current_target.demandAz, current_target.demandEl, rot_tel=current_target.demandRot, wait_dome=True)
await asyncio.sleep(2)
await latiss.take_engtest(16)

In [ ]:
# note demands velocities are in deg/s
0.006491433512394764*3600

In [ ]:
# non siderial tests

In [ ]:
# Slew to target and setup instrument
await atcs.slew_object("HR 2934", rot_type=RotType.PhysicalSky, rot=-90, ddec=10)
await latiss.take_object(12)

In [ ]:
await atcs.stop_tracking()

In [ ]:
# Slew to target and setup instrument
await atcs.slew_object("HD 82446", rot_type=RotType.PhysicalSky, rot=-90)
await latiss.setup_atspec(filter='RG610', grating='empty_1')

In [ ]:
current_target = await atcs.rem.atptg.tel_currentTargetStatus.next(flush=True, timeout=4) # isn't publishing alt-az!
print(current_target)

In [ ]:
# Slew to target and setup instrument
await atcs.slew_object("HD115782", rot_type=RotType.PhysicalSky, rot=-110)
await latiss.take_object(5)
# ra_offset=60 # RA offset
# dec_offset = 0
# await atcs.offset_radec(ra_offset, dec_offset)
# await latiss.take_object(2)

In [ ]:
await atcs.rem.ataos.cmd_applyFocusOffset.set_start(offset=-0.8)

In [ ]:
await latiss.take_object(5)

In [ ]:
await latiss.take_object(10)

In [ ]:
await atcs.offset_radec(ra_offset, dec_offset)
await latiss.take_object(2)

In [ ]:
await atcs.offset_xy(x=-60, y=-100)

In [ ]:
# Offset in RA, turn off tracking, start long exposure
ra_offset=6*60 # RA offset to the WEST
dec_offset = 0
dec=-89 # degrees
dra= -1/np.cos(np.abs(dec*np.pi/180)) # second/sec
dra=-9
await atcs.slew_object("HD 90105", rot_type=RotType.PhysicalSky, rot=-110, ddec=0, dra = dra)
await atcs.offset_xy(x=-70, y=0)
await atcs.offset_radec(ra_offset, dec_offset)
await latiss.take_engtest(60)

In [ ]:
dra

In [ ]:
# Offset in RA, turn off tracking, start long exposure
# Faster rate
ra_offset=10*60 # RA offset to the WEST
dec_offset = 0
dec=9.0 # degrees
dra= -2*np.cos(dec*np.pi/180) # second/sec
await atcs.slew_object("HD115782", rot_type=RotType.PhysicalSky, rot=-110, ddec=0, dra = dra)
await atcs.offset_xy(x=-60, y=-100)
await atcs.offset_radec(ra_offset, dec_offset)
await latiss.take_engtest(11)

In [ ]:
await atcs.stop_tracking()

In [ ]:
360*3600 / (24*60*60)

In [ ]:
# get current telescope Alt/Az position
# need to calculate it from current RA/DEC because of a timestamp issue
az = Angle(start_az, unit=u.deg)
el = Angle(start_el, unit=u.deg)

print(f'orig az {az} and el {el}')
time_data = await atcs.rem.atptg.tel_timeAndDate.next(flush=True, timeout=2)
# This should be TAI and not UTC... so will be 37s off system clock seconds
curr_time_atptg = Time(time_data.mjd, format="mjd")

coord_frame_AltAz = AltAz(location=location, obstime=curr_time_atptg)
coord_frame_radec = ICRS()
coord_azel = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec = coord_azel.transform_to(coord_frame_radec)
print('Current Position is: \n {}'.format(coord_azel))
print('Current Position is: \n {}'.format(ra_dec))

# get RA/DEC of target position
az = Angle(start_az+d_az, unit=u.deg)
el = Angle(start_el+d_el, unit=u.deg)
print(f'target az {az} and el {el}')
coord_azel_target = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec_target = coord_azel_target.transform_to(coord_frame_radec)
print('Target Position is: \n {}'.format(coord_azel_target))
print('Target Position is: \n {}'.format(ra_dec_target))

In [ ]:
# get current telescope Alt/Az position
# need to calculate it from current RA/DEC because of a timestamp issue
az = Angle(start_az, unit=u.deg)
el = Angle(start_el, unit=u.deg)
print(f'orig az {az} and el {el}')
time_data = await atcs.rem.atptg.tel_timeAndDate.next(flush=True, timeout=2)
# This should be TAI and not UTC... so will be 37s off system clock seconds
curr_time_atptg = Time(time_data.mjd, format="mjd")

coord_frame_AltAz = AltAz(location=location, obstime=curr_time_atptg)
coord_frame_radec = ICRS()
coord_azel = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec = coord_azel.transform_to(coord_frame_radec)
print('Current Position is: \n {}'.format(coord_azel))
print('Current Position is: \n {}'.format(ra_dec))

# get RA/DEC of target position
az = Angle(start_az+d_az, unit=u.deg)
el = Angle(start_el+d_el, unit=u.deg)
print(f'target az {az} and el {el}')
coord_azel_target = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec_target = coord_azel_target.transform_to(coord_frame_radec)
print('Target Position is: \n {}'.format(coord_azel_target))
print('Target Position is: \n {}'.format(ra_dec_target))

In [ ]:
#send to starting position
# Note that the wait_settle=False is used to remove sleeps
# that are no longer required in the ATCS now that ATMCS

await attcs.slew_icrs(ra=str(ra_dec.ra), dec=str(ra_dec.dec), rot_pa=0.0,
                      slew_timeout=240., stop_before_slew=False, wait_settle=False)


print('track for 2s')
await asyncio.sleep(2)
# take a quick image to get some header data
await latiss.take_engtest(exptime=1)

print('Starting to Slew to target')
start_time = time.time()
await attcs.slew_icrs(ra=str(ra_dec_target.ra), dec=str(ra_dec_target.dec),
                      rot_pa=d_rot, slew_timeout=240., 
                      stop_before_slew=False, wait_settle=False)
end_time = time.time()
print('Time to slew is {} [s]'.format(end_time-start_time))
await latiss.take_engtest(exptime=1)

# Print current time so it's easier to find info in EFD
current_time = datetime.now().strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

In [ ]:
# Now you'll want run the test but going to the original position!

print('Starting to Slew to original target')
start_time = time.time()
await attcs.slew_icrs(ra=str(ra_dec_target.ra), dec=str(ra_dec_target.dec),
                      rot_pa=d_rot, slew_timeout=240., 
                      stop_before_slew=False, wait_settle=False)
end_time = time.time()
print('Time to slew is {} [s]'.format(end_time-start_time))
await latiss.take_engtest(exptime=1)

# Print current time so it's easier to find info in EFD
current_time = datetime.now().strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

In [ ]:
# For shutdown of system
await attcs.stop_tracking()

In [ ]:
# turn off corrections
tmp = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)